In [ ]:
from pathlib import Path
from utils.get_input import get_input
import numpy as np


inp = Path("./test_input").read_text()
inp = get_input(17)
inp;

In [ ]:
regs, instrs = inp.split("\n\n")
reg_a, reg_b, reg_c = regs = [int(x.split(": ")[1]) for x in regs.split("\n")]
instrs = [int(x) for x in instrs.lstrip("Program: ").split(",")]

regs, instrs;

In [3]:
ip = 0
reg_a, reg_b, reg_c = regs  # Reset registers
out = []
print(regs, instrs)


def combo(operand):
    match operand:
        case 0 | 1 | 2 | 3:
            return operand
        case 4:
            return reg_a
        case 5:
            return reg_b
        case 6:
            return reg_c
        case 7:
            raise ValueError("Invalid operand")


while ip < len(instrs):
    # print(ip, instrs[ip], instrs[ip + 1], "Regs: ", [reg_a, reg_b, reg_c])

    match instrs[ip]:
        case 0:  # adv
            reg_a = int(reg_a / 2 ** combo(instrs[ip + 1]))
        case 1:  # bxl
            reg_b = reg_b ^ instrs[ip + 1]
        case 2:  # bst
            reg_b = combo(instrs[ip + 1]) % 8
        case 3:  # jnz
            if reg_a != 0:
                ip = instrs[ip + 1] - 2
        case 4:  # bxc
            reg_b = reg_b ^ reg_c
        case 5:  # out
            out.append(combo(instrs[ip + 1]) % 8)
        case 6:  # bdv
            reg_b = int(reg_a / 2 ** combo(instrs[ip + 1]))
        case 7:  # cdv
            reg_c = int(reg_a / 2 ** combo(instrs[ip + 1]))
    ip += 2

print(",".join(map(str, out)))

[28422061, 0, 0] [2, 4, 1, 1, 7, 5, 1, 5, 4, 2, 5, 5, 0, 3, 3, 0]
3,6,7,0,5,7,3,1,4


In [4]:
from numba import njit


@njit
def fast_func2(i):
    a = i
    bs = []
    while a > 0:
        b = (a % 8) ^ 1
        c = a // (2**b)
        b = (b ^ 5) ^ c % 8
        bs.append(b)
        a = a // 8
    return bs


def depth_first_find_i(instrs, num, p):
    ins = instrs[p]
    for i in range(1, 8):
        if p == 0 and 7 == 0:
            continue

        new_num = num + i * (8 ** (15 - p))
        tst = fast_func2(new_num)[-(1 + p)]
        print(f"Ins: {ins}, Num: {oct(new_num)}, Tst: {tst}")
        # num = new_num
        print(oct(num))
        if tst != ins:
            continue
        if p == 15:
            print("Found", oct(new_num))
            return True, new_num
        else:
            found, i = depth_first_find_i(instrs, new_num, p + 1)
            if found:
                return True, i
    else:
        print("No match")
        return False, 0


num_start = 0o1000000000000000

res, i = depth_first_find_i(instrs[::-1], num_start, 0)

if res:
    print(oct(i))
    print(i)


Ins: 0, Num: 0o2000000000000000, Tst: 6
0o1000000000000000
Ins: 0, Num: 0o3000000000000000, Tst: 7
0o1000000000000000
Ins: 0, Num: 0o4000000000000000, Tst: 0
0o1000000000000000
Ins: 3, Num: 0o4100000000000000, Tst: 4
0o4000000000000000
Ins: 3, Num: 0o4200000000000000, Tst: 2
0o4000000000000000
Ins: 3, Num: 0o4300000000000000, Tst: 7
0o4000000000000000
Ins: 3, Num: 0o4400000000000000, Tst: 1
0o4000000000000000
Ins: 3, Num: 0o4500000000000000, Tst: 3
0o4000000000000000
Ins: 3, Num: 0o4510000000000000, Tst: 4
0o4500000000000000
Ins: 3, Num: 0o4520000000000000, Tst: 3
0o4500000000000000
Ins: 0, Num: 0o4521000000000000, Tst: 4
0o4520000000000000
Ins: 0, Num: 0o4522000000000000, Tst: 4
0o4520000000000000
Ins: 0, Num: 0o4523000000000000, Tst: 3
0o4520000000000000
Ins: 0, Num: 0o4524000000000000, Tst: 2
0o4520000000000000
Ins: 0, Num: 0o4525000000000000, Tst: 4
0o4520000000000000
Ins: 0, Num: 0o4526000000000000, Tst: 0
0o4520000000000000
Ins: 5, Num: 0o4526100000000000, Tst: 4
0o45260000000000